# National Vaccine Injury Compensation Program (vicp)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

cc = plt.rcParams["axes.prop_cycle"].by_key()["color"]

from vicp import vicp

## Load vicp report

Should download the vicp report from the hrse.gov site. (Only vetted for December 2020 report, unfortunately tabula is not smart enough to autoscrap the data and needed a template.)

In [ ]:
data = vicp()

## vicp tables

The vicp object has five dataframes for each table in the report. They dataframes are named: `adjudications`, `petitions`,  `annual_petitions`, `annual_adjudications`, and `awards`.

In [ ]:
data.awards

# Vaccine Plots

Here is a bar chart of the number of petitions filed relating to each vaccine, see report document for details.

In [ ]:
x = data.petitions.Vaccine
y1 = data.petitions.Death
y2 = data.petitions.Injury

report = [y1+y2 != 0][0]
x, y1, y2 = x[report], y1[report], y2[report]

order = np.argsort(y1+y2)
x, y1, y2 = x[order], y1[order], y2[order]


fig, ax = plt.subplots()
b1 = ax.bar(x, y1, color=cc[3], label='Deaths')
b2 = ax.bar(x, y2, bottom=y1, color=cc[0], label='Injuries')
ax.set_yscale('log')
ax.set_xlabel('Vaccine', labelpad=20)
ax.set_ylabel('Adverse Incidents')
ax.set_title('Petitions between 1988-2020')
ax.legend()
plt.draw()
for rect, label, height in zip(b2, ax.get_xticklabels(), y1+y2):
    label = label._text.strip()
    ax.annotate('{}'.format(label),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, -3), textcoords="offset points", rotation=90,
                ha='center', va='top', fontsize=12)
ax.get_xaxis().set_ticks([])
fig.tight_layout();

### Vaccine adverse rate

The next bar chart shows an inferred adverse incident rate for each vaccine. The bounds on the rates are arguable, as not all adverse incidents suffered will be process by the vicp, and on the other hand not all petitions have merit. Regardless, these rates should be representative of the rate of adverse effects for each vaccine.

*Deaths and Injuries are estimated from the larger 1988-2020 petition data.

In [ ]:
df = data.adjudications.merge(data.petitions, how='inner', on=['Vaccine'])

x = df.Vaccine.values
y = np.divide(df.Totals_x, df.Doses, out=np.zeros(len(df.Totals_x)),
              where=(df.Doses!=0)).values
f = (df.Death/df.Totals_y).values

report = (y != 0)
x, y, f = x[report], y[report], f[report]

order = np.argsort(y)
x, y, f = x[order], y[order], f[order]

fig, ax = plt.subplots()
bar = ax.bar(x, y, color=cc[0], label=r'Injury$^*$')
ax.bar(x, f*y, color=cc[3], label=r'Death$^*$')

ax.set_xlabel('Vaccine', labelpad=20)
ax.set_ylabel('Adverse Rate')
ax.set_title('Vaccine Adverse Petition Rate per Dose\nBetween 2006 and 2020', pad=10)
ax.legend(fontsize=18)
ax.set_ylim([1e-8, 1e-4])
ax.set_yscale('log')
plt.draw()
for rect, label, height in zip(bar, ax.get_xticklabels(), y):
    label = label._text.strip()
    if height == 0:
        continue
    ax.annotate('{}'.format(label),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, -3), textcoords="offset points", rotation=90,
                ha='center', va='top', fontsize=12)

ylims = ax.get_ylim()
outof = np.logspace(-9, -2, 8)
outof_text = [f'1 out of {round(1/x):,d}' for x in outof]
for of, text in zip(outof, outof_text):
    ax.axhline(of, c=cc[7], zorder=0, ls='--', alpha=0.4, lw=2)
    shift = 1.15
    if of < y[5]/shift or of > ylims[1]/shift or of < ylims[0]*shift:
        continue
    ax.text(0, shift*of, text, color=cc[7], fontsize=10, zorder=0)

ax.get_xaxis().set_ticks([])
fig.tight_layout()

fig.savefig('adverse_rate.png', facecolor=fig.get_facecolor(), edgecolor='none',
            bbox_inches='tight')

The next plot is the rate at which petitions are filed with the vicp

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.annual_petitions['Fiscal Year'], data.annual_petitions['Total'],
        '-', lw=4, c=cc[7])
ax.plot(data.annual_petitions['Fiscal Year'], data.annual_petitions['Total'],
        'P', ms=15, c=cc[3])
ax.set_title('Petitions Filed (Annual)')
ax.set_xlabel('Year')
ax.set_ylabel('Petitions Filed')
fig.autofmt_xdate()
fig.tight_layout()

The next two plots show the average compenstion of a petition that was deemed to have merit and requiring compensation from the fund. The first plot is the average compensation to the victim's family and the second plot is the average fees collected by lawyers represented the victims.

In [ ]:
x = data.awards['Year']
y1 = (data.awards['Attorney Fees']
      +data.awards['Interim Fees'])/data.awards['Awards']
y2 = data.awards['Amount']/data.awards['Awards']-y1

fig, ax = plt.subplots()
ax.plot(x, y2, ':', c=cc[5])
ax.plot(x, y2, 'x', ms=10, mew=5, c=cc[8])
ax.set_xlabel('Year')
ax.set_ylabel('Compensation')
ax.set_title("Average Compensation to Victim's Family", pad=10)
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)
fig.autofmt_xdate()
fig.tight_layout()
fig.savefig('victim_compensation.png', facecolor=fig.get_facecolor(),
            edgecolor='none', bbox_inches='tight')

fig, ax = plt.subplots()
ax.plot(x, y1, ':', c=cc[9])
ax.plot(x, y1, 'x', ms=10, mew=5, c=cc[6])
ax.set_xlabel('Year')
ax.set_ylabel('Fees')
ax.set_title("Average Legal Fees\n(Victim Compensated)", pad=10)
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick)
fig.autofmt_xdate()
fig.tight_layout()
fig.savefig('legal_fees.png', facecolor=fig.get_facecolor(), edgecolor='none',
            bbox_inches='tight')